## Appendix
PCA and clustering using scRNA-seq data and BANKSY markers

### data

#### Matched Single-Cell RNA Sequencing

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.io import mmread
# from scipy import sparse
import anndata as ad
from sklearn.preprocessing import StandardScaler

data_path = Path("../data/mouse_hypothalamus/singlecell/")
mtx_path = data_path / "GSE113576_matrix.mtx"
barcodes_path = data_path / "GSE113576_barcodes.tsv"
genes_path = data_path / "GSE113576_genes.tsv"
meta_path = data_path / "aau5324_Moffitt_Table-S1.xlsx"

In [ ]:
X = mmread(mtx_path).tocsr()
cell_ids = pd.read_csv(barcodes_path, sep="\t", header=None)[0].values
gene_names = pd.read_csv(genes_path, sep="\t", header=None)[1].values

adata = ad.AnnData(X=X.T)  # cell x gene
adata.var_names = gene_names
adata.obs_names = cell_ids
adata.var_names_make_unique()

# filter the unstable and ambiguous cells
meta = pd.read_excel(meta_path).rename(columns={
    "Cell name": "Cell_name",
    "Sex": "Sex",
    "Replicate number": "Rep",
    "Cell class (determined from clustering of all cells)": "Cell_class",
    "Non-neuronal cluster (determined from clustering of all cells)": "Non_neuronal_cluster",
    "Neuronal cluster (determined from clustering of inhibitory or excitatory neurons)": "Neuronal_cluster"
})

cell_class = {'Mature oligodendrocyte': 'OD mature'}
meta = meta.set_index("Cell_name")
meta = meta.loc[meta["Cell_class"].isin(cell_class.keys())]
adata = adata[adata.obs_names.isin(meta.index)].copy()
adata.obs = meta.loc[adata.obs_names, ["Cell_class"]].copy()
adata.obs["Cell_class"] = adata.obs["Cell_class"].map(cell_class)

# Filter out cells where mitochondrial RNA fraction is >= 20%
mt_mask = adata.var_names.str.startswith("mt")
mt_fraction = np.array(adata[:, mt_mask].X.sum(axis=1)).flatten() / (
    np.array(adata.X.sum(axis=1)).flatten() + 1e-6
)
adata = adata[mt_fraction < 0.2, :].copy()

# keep cells with more than 1,000 detected genes
nonzero_counts = np.array((adata.X != 0).sum(axis=1)).flatten()
adata = adata[nonzero_counts > 1000, :].copy()

# filter out "blank" controls
blank_mask = ~adata.var_names.str.startswith("Blank")
adata = adata[:, blank_mask].copy()

sc_total = np.array(adata.X.sum(axis=1)).flatten()
adata.X = adata.X.multiply(10_000 / sc_total[:, None])  # normalize
adata.X = adata.X.log1p()  # log1p
adata.X = StandardScaler(with_mean=False).fit_transform(adata.X)

#### Marker Genes

differentially expressed genes identified by BANKSY

In [ ]:
# all differentially expressed genes
DE_genes = ['Mlc1', 'Dgkk', 'Cbln2', 'Syt4', 'Gad1', 'Plin3', 'Gnrh1', 'Sln', 'Gjc3', 'Mbp', 'Lpar1', 'Trh', 'Ucn3', 'Cck']
# DE_genes_gm: 7
DE_genes_MOD2 = ['Mlc1', 'Dgkk', 'Cbln2', 'Syt4', 'Gad1', 'Plin3', 'Gnrh1', 'Sln', 'Gjc3']
# DE_genes_wm: 8
DE_genes_MOD1 = ['Mbp', 'Lpar1', 'Trh', 'Ucn3', 'Cck']

In [ ]:
sc_data = adata.to_df()
sc_DE_MOD2_df = sc_data[DE_genes_MOD2]
sc_DE_MOD1_df = sc_data[DE_genes_MOD1]
sc_DE = pd.concat([sc_DE_MOD2_df, sc_DE_MOD1_df], axis=1)

### BANKSY method: top 25 correlated genes for each marker

In [ ]:
# Find genes most correlated with a given gene.

from scipy.stats import spearmanr

def find_corr_genes(gene, sc_data):
    cor = pd.Series(index=sc_data.index, dtype=np.float64)
    x = pd.to_numeric(sc_data.loc[gene].values.flatten(), errors='coerce')
    
    for i, other_gene in enumerate(sc_data.index):
        if other_gene.startswith("Blank"):
            continue
        if other_gene == 'Cell_class':
            continue
        
        y = pd.to_numeric(sc_data.loc[other_gene].values.flatten(), errors='coerce')
        
        if len(x) != len(y):
            continue

        cor[other_gene], p_value = spearmanr(x, y)

    return cor.sort_values(ascending=False)


def process_related_genes(DE_genes, sc_data):
    corr_genes = []
    for gene in DE_genes:
        corr_g = find_corr_genes(gene, sc_data)
        top_25_genes = corr_g.iloc[:26].index.tolist()
        corr_genes.extend(top_25_genes)
    
    return corr_genes

In [ ]:
top25_corr_genes = process_related_genes(DE_genes, sc_data.T)

In [ ]:
DE_scRNA_data =  sc_data[DE_genes]
DE_corr_scRNA_data =  sc_data[top25_corr_genes]

### PCA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
import umap

In [ ]:
data = DE_scRNA_data.copy()

pca_full = PCA(n_components=14, random_state=42)
pca_full_result = pca_full.fit_transform(data)
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

In [ ]:
# (1) Explained Variance Ratio
plt.figure(figsize=(8,6), dpi=600)

plt.plot(np.arange(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='--', color='mediumvioletred')
plt.xlabel("Number of Principal Components")
plt.ylabel("Explained Variance Ratio")
# plt.title("Explained Variance Ratio by PCs")
plt.title("Only MOD Markers")

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.show()

In [ ]:
# (2) Cumulative Explained Variance
plt.figure(figsize=(8,6), dpi=600)
# plt.hlines(y=1, xmin=0, xmax=14, linestyles='--', colors='lightblue')
plt.plot(np.arange(1, len(explained_variance) + 1), cumulative_variance, marker="o", linestyle="-", color='mediumvioletred')
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
# plt.title("Cumulative Explained Variance Ratio")

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.show()

In [ ]:
pca_full = PCA(n_components=5, random_state=42)
pca_full_result = pca_full.fit_transform(data)
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# UMAP on PCA-Reduced Data
umap_model = umap.UMAP(n_components=2)
umap_result = umap_model.fit_transform(pca_full_result)

# (3) UMAP Clustering (Using Specified n_PCs)
plt.figure(figsize=(8,6), dpi=600)

scatter_umap = plt.scatter(umap_result[:, 0], umap_result[:, 1], alpha=0.7, s=11, edgecolors='none', color='mediumvioletred')
# plt.title(f"UMAP using n_PCs = 5")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.show()

In [ ]:
data = DE_corr_scRNA_data.copy()

pca_full = PCA(n_components=364, random_state=42)
pca_full_result = pca_full.fit_transform(data)
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

In [ ]:
# (1) Explained Variance Ratio
plt.figure(figsize=(8,6), dpi=600)

plt.plot(np.arange(1, len(explained_variance) + 1), explained_variance, marker='.', linestyle='--', color='mediumvioletred')
plt.xlabel("Number of Principal Components")
plt.ylabel("Explained Variance Ratio")
# plt.title("Explained Variance Ratio by PCs")
plt.title("MOD Markers and Correlated Genes")

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.show()

In [ ]:
# (2) Cumulative Explained Variance
plt.figure(figsize=(8,6), dpi=600)
# plt.hlines(y=1, xmin=0, xmax=364, linestyles='--', colors='lightblue')
plt.plot(np.arange(1, len(explained_variance) + 1), cumulative_variance, marker=".", linestyle="-", color='mediumvioletred')
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
# plt.title("Cumulative Explained Variance Ratio")

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.show()

In [ ]:
pca_full = PCA(n_components=5)
pca_full_result = pca_full.fit_transform(data)
# UMAP on PCA-Reduced Data
umap_model = umap.UMAP(n_components=2)
umap_result = umap_model.fit_transform(pca_full_result)

# (3) UMAP Clustering (Using Specified n_PCs)
plt.figure(figsize=(8,6), dpi=600)

scatter_umap = plt.scatter(umap_result[:, 0], umap_result[:, 1], alpha=0.7, s=11, edgecolors='none', color='mediumvioletred')
# plt.title(f"UMAP using n_PCs = 5")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")

ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.show()

### k-means clustering

In [ ]:
from sklearn.cluster import KMeans
from matplotlib.colors import LinearSegmentedColormap

_BIH_CMAP = LinearSegmentedColormap.from_list(
    "BIH",
    [
        "mediumvioletred",
        "violet",
        "powderblue",
        "powderblue",
    ][::-1],
)

_BIH_CMAP_re = LinearSegmentedColormap.from_list(
    "BIH",
    [
        "powderblue",
        "powderblue",
        "violet",
        "mediumvioletred",
    ][::-1],
)

In [ ]:
def kmeans_clustering(data_for_clustering, k=2, n_PCs=5):
    data_for_clustering = data_for_clustering.copy()
    # Step 1: KMeans Clustering
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=2)
    kmeans.fit(data_for_clustering)
    
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
  
    # PCA for explained variance ratio
    pca_full = PCA(n_components=n_PCs, random_state=42)
    pca_full_result = pca_full.fit_transform(data_for_clustering)
    
    # UMAP on PCA-Reduced Data
    umap_model = umap.UMAP(n_components=2)
    umap_result = umap_model.fit_transform(pca_full_result)
    
    # Visualization
    plt.figure(figsize=(8,6), dpi=600)
    
    # (4) UMAP Clustering (Using Specified n_PCs)
    scatter_umap = plt.scatter(umap_result[:, 0], umap_result[:, 1], c=labels, cmap=_BIH_CMAP_re, alpha=0.6, s=11, edgecolors='none')
    # plt.title(f"Only MOD markers \n K-means Clustering")
    plt.title(f"K-means Clustering")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    # plt.legend(handles=scatter_umap.legend_elements()[0], labels=[f'Cluster {i}' for i in range(k)])
    
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    plt.show()
    
    return labels, centroids

In [ ]:
OD_k_labels, OD_centroids = kmeans_clustering(DE_scRNA_data, k=2, n_PCs=2)

In [ ]:
OD_k_labels, OD_centroids = kmeans_clustering(DE_corr_scRNA_data, k=2, n_PCs=5)

### leiden clustering

In [ ]:
from sklearn.neighbors import NearestNeighbors
import scipy.sparse as sp
import igraph as ig
import leidenalg as la

In [ ]:
def leiden_clustering(data_for_clustering, k=15, resolution=0.5, n_PCs=5):
    """
    Perform Leiden clustering and visualize the results in PCA and UMAP spaces.
    
    Parameters:
    - data_for_clustering (numpy.ndarray or pandas.DataFrame): Data matrix (cells x genes)
    - k (int): Number of neighbors for kNN graph (default: 15)
    - resolution (float): Resolution parameter for Leiden clustering (default: 0.5)
    - n_PCs (int or None): Number of principal components to use for UMAP (default: None, auto-detect)
    
    Returns:
    - cluster_assignments (numpy.ndarray): Array of cluster labels
    """
    # Input validation
    if k < 1 or not isinstance(k, int):
        raise ValueError("k must be a positive integer.")
    if resolution <= 0:
        raise ValueError("Resolution must be greater than 0.")

    # Find k-nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(data_for_clustering)
    knn_graph = nbrs.kneighbors_graph(data_for_clustering, mode='connectivity')

    # Convert sparse matrix to igraph
    sources, targets = knn_graph.nonzero()
    edges = list(zip(sources, targets))
    graph = ig.Graph(edges=edges, directed=False)

    # Add weights if desired
    weights = knn_graph.data
    graph.es['weight'] = weights

    # Perform Leiden clustering
    partition = la.find_partition(graph, partition_type=la.RBConfigurationVertexPartition, resolution_parameter=resolution)
    cluster_assignments = np.array(partition.membership)

    # PCA for explained variance ratio
    pca_full = PCA(n_components=n_PCs, random_state=42)
    pca_full_result = pca_full.fit_transform(data_for_clustering)
    
    # UMAP on PCA-Reduced Data
    umap_model = umap.UMAP(n_components=2)
    umap_result = umap_model.fit_transform(pca_full_result)
    
    # Visualization
    plt.figure(figsize=(8,6), dpi=600)
    
    scatter_umap = plt.scatter(umap_result[:, 0], umap_result[:, 1], c=cluster_assignments, cmap=_BIH_CMAP, alpha=0.6, s=11, edgecolors='none')
    plt.title(f"Leiden Clustering")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # plt.legend(handles=scatter_umap.legend_elements()[0], labels=[f'Cluster {i}' for i in np.unique(cluster_assignments)])
    plt.show()
    
    return cluster_assignments


In [ ]:
OD_leiden_labels = leiden_clustering(DE_scRNA_data, k=50, resolution=0.02, n_PCs=2)

In [ ]:
OD_leiden_labels = leiden_clustering(DE_corr_scRNA_data, k=50, resolution=0.3, n_PCs=5)

### hierarchical clustering


In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
def hierarchical_clustering(data_for_clustering, k=2, n_PCs=5):
    # Hierarchical Clustering
    # ‘ward’ minimizes the variance of the clusters being merged.
    hierarchical = AgglomerativeClustering(n_clusters=k)
    labels = hierarchical.fit_predict(data_for_clustering)
    
    # PCA for explained variance ratio
    pca_full = PCA(n_components=n_PCs, random_state=42)
    pca_full_result = pca_full.fit_transform(data_for_clustering)
    # UMAP on PCA-Reduced Data
    umap_model = umap.UMAP(n_components=2)
    umap_result = umap_model.fit_transform(pca_full_result)
    
    
    # Visualization
    plt.figure(figsize=(8,6), dpi=600)
    
    scatter_umap = plt.scatter(umap_result[:, 0], umap_result[:, 1], c=labels, cmap=_BIH_CMAP, alpha=0.6, s=11, edgecolors='none')
    plt.title(f"Hierarchical Clustering")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # plt.legend(handles=scatter_umap.legend_elements()[0], labels=[f'Cluster {i}' for i in np.unique(labels)])
    plt.show()
    
    return labels

In [ ]:
OD_h_labels = hierarchical_clustering(DE_scRNA_data, k=2, n_PCs=2)

In [ ]:
OD_h_labels = hierarchical_clustering(DE_corr_scRNA_data, k=2, n_PCs=5)